In [1]:
from haystack import Pipeline
from haystack.nodes import PreProcessor, DensePassageRetriever
from haystack.document_stores import ElasticsearchDocumentStore
from haystack import Document
import texthero as hero
import os
import pandas as pd
import glob

In [2]:
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

In [3]:
document_store = ElasticsearchDocumentStore(
        host=host,
        username="",
        password="",
        index="document",
        similarity="dot_product",
        embedding_dim=768,
    )

/Users/shreygupta/opt/anaconda3/envs/nlp/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [18]:
document_store.get_document_count()

14584

In [5]:
file_list_level1 = glob.glob("../data/pratt.duke.edu/*.txt", recursive=False)
file_list_level2 = glob.glob("../data/pratt.duke.edu/*/*.txt", recursive=False)
file_list_level3 = glob.glob("../data/pratt.duke.edu/*/*/*.txt", recursive=False)
file_list_level4 = glob.glob("../data/pratt.duke.edu/*/*/*/*.txt", recursive=False)
file_list_level5 = glob.glob("../data/pratt.duke.edu/*/*/*/*/*.txt", recursive=False)

In [6]:
len(file_list_level1) + len(file_list_level2) + len(file_list_level3) + len(file_list_level4) + len(file_list_level5)

3189

In [7]:
i = 0
for file in file_list_level5:
    with open(file) as f:
            text = f.read()
    if i == 0:
        df = pd.DataFrame({"text": [text]})
    else:
        df = df.append({"text": text}, ignore_index=True)
    i += 1

/var/folders/zq/4t83990x3sj657j051pwmbn40000gn/T/ipykernel_46806/3269226494.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"text": text}, ignore_index=True)
/var/folders/zq/4t83990x3sj657j051pwmbn40000gn/T/ipykernel_46806/3269226494.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"text": text}, ignore_index=True)
/var/folders/zq/4t83990x3sj657j051pwmbn40000gn/T/ipykernel_46806/3269226494.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"text": text}, ignore_index=True)
/var/folders/zq/4t83990x3sj657j051pwmbn40000gn/T/ipykernel_46806/3269226494.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

In [8]:
df

,text
0,\n\nSkip to main content\n\n * Departments & Centers\n * Overview\n *...
1,\n\nSkip to main content\n\n * Departments & Centers\n * Overview\n *...
2,\n\nSkip to main content\n\n * Departments & Centers\n * Overview\n *...
3,\n\nSkip to main content\n\n * Departments & Centers\n * Overview\n *...
4,\n\nSkip to main content\n\n * Departments & Centers\n * Overview\n *...
...,...
658,\n\nSkip to main content\n\n * Departments & Centers\n * Overview\n *...
659,\n\nSkip to main content\n\n * Departments & Centers\n * Overview\n *...
660,\n\nSkip to main content\n\n * Departments & Centers\n * Overview\n *...
661,\n\nSkip to main content\n\n * Departments & Centers\n * Overview\n *...


In [9]:
custom_pipeline = [hero.preprocessing.remove_whitespace,
                    hero.preprocessing.remove_angle_brackets,
                    hero.preprocessing.remove_html_tags,
                    hero.preprocessing.remove_urls]

In [10]:
df["cleaned_text"] = hero.clean(df["text"], custom_pipeline)

In [11]:
df["cleaned_text"] = df["cleaned_text"].str.replace("Skip to main content * Departments & Centers * Overview * Biomedical Engineering * Civil & Environmental Engineering * Electrical & Computer Engineering * Mechanical Engineering & Materials Science * Institute for Enterprise Engineering * Alumni & Parents * Overview * Alumni * Parents * Giving * Board of Visitors * Our History * Email Newsletter * Meet the Team * Corporate Partners * Overview * Partners & Sponsors * Data Science & AI Industry Affiliates * Connect With Students * Recruiting Our Students * Sponsored Research * TechConnect Career Networking * Apply * Careers * Directory * Undergraduate * 1. For Prospective Students 1. Majors & Minors 2. Certificates 3. General Degree Requirements 4. 4+1: BSE+Master's Degree 5. Campus Tours 6. How to Apply 2. First-Year Design 3. Student Entrepreneurship 4. Undergraduate Research 5. Where Our Undergrads Go 6. Diversity, Equity & Inclusion 7. For Current Students 1. The First Year 2. Advising 3. Student Clubs & Teams 4. Graduation with Distinction 5. Internships 6. Policies & Procedures * Graduate * 1. For Prospective Students 1. PhD Programs 2. Master's Degrees 3. Online Specializations, Certificates and Short Courses 4. Admissions Events 5. How to Apply 2. For Admitted Students 3. Diversity, Equity & Inclusion 1. Bootcamp for Applicants 2. Recruiting Incentives 4. For Current Grad Students 1. Graduate Student Programs & Services * Faculty & Research * 1. Faculty 1. Faculty Profiles 2. New Faculty 3. Awards and Recognition 4. NAE Members 2. Research 1. Signature Research Themes 2. Recent External Funding Awards 3. Faculty Entrepreneurship 4. Duke Engineering Discoveries * About * 1. Dean's Welcome 2. Campus & Tours 3. Facts & Rankings 4. Diversity, Equity & Inclusion 5. Service to Society 6. Entrepreneurship 7. Governance 8. News & Media 1. Latest News 2. Podcast 3. Email Newsletter 4. Publications 5. Media Coverage 6. Public Health Information 9. Events 1. Events Calendar 2. Academic Calendar 3. Commencement 10. Art @ Duke Engineering", "", regex=False)

In [12]:
list_of_docs = []
for index, row in df.iterrows():
    doc = Document(row["cleaned_text"])
    list_of_docs.append(doc)

In [13]:
preprocessor = PreProcessor(
        clean_whitespace=True,
        clean_header_footer=True,
        clean_empty_lines=True,
        split_by="word",
        split_length=250,
        split_overlap=10,
        split_respect_sentence_boundary=True
)

# split documents into smaller chunks
preprocessed_docs = preprocessor.process(documents=list_of_docs)


Preprocessing:   0%|          | 0/663 [00:00<?, ?docs/s]

We found one or more sentences whose word count is higher than the split length.
Document 25cb5bb7fb7cc8803a203c63cd97576a is 11890 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time.
Document b2522560145a86c3e3e223edb0c7c454 is 11718 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time.
Document b971b868b8cbb2f625469977e2c199f5 is 18876 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time.
Document 963423f5e0ae23a73f8c22829d1935d4 is 12595 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to preve

In [14]:
len(preprocessed_docs)

5060

In [15]:
# write documents to document store
document_store.write_documents(preprocessed_docs)

/Users/shreygupta/opt/anaconda3/envs/nlp/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [16]:
retriever = DensePassageRetriever(
        document_store=document_store,
        query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
        passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base"
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [17]:
document_store.update_embeddings(retriever, update_existing_embeddings=False)

Updating embeddings:   0%|          | 0/5056 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/5056 [00:00<?, ? Docs/s]